# ROS Package

ROS: melodic, currently we are using python 2 with melodic.

## Setup

This will take a couple minutes

In [ ]:
#@title Install ROS {display-mode: "form"}
!sudo sh -c 'echo "deb http://packages.ros.org/ros/ubuntu $(lsb_release -sc) main" > /etc/apt/sources.list.d/ros-latest.list'
!sudo apt-key adv --keyserver 'hkp://keyserver.ubuntu.com:80' --recv-key C1CF6E31E6BADE8868B172B4F42ED6FBAB17C654
!sudo apt update
!sudo apt install ros-melodic-ros-base

### Setup Python Path

So that we could use rospy, rosbag etc in the notebook.

In [ ]:
#@title setup python path {display-mode: "form"}
import sys
import os

sys.path.append('/opt/ros/melodic/lib/python2.7/dist-packages/')
#sys.path.append('/opt/ros/melodic/lib/python3.6/dist-packages/')
print(sys.path)

os.environ['PATH'] += ':/opt/ros/melodic/bin'
print(os.environ['PATH'])

In [ ]:
!sudo apt install ros-melodic-cv-bridge ros-melodic-ros-numpy
!sudo apt install python-catkin-tools
!sudo apt install python-rosdep python-rosinstall python-rosinstall-generator python-wstool build-essential
!sudo apt install ros-melodic-rosbridge-server
!sudo apt install python-rospkg
!pip install rospkg

This will create a **my_package** folder which contains a **package.xml** and a **CMakeLists.txt**, which have been partially filled out with the information we gave catkin_create_pkg.

## A. Catkin Workspace

Catkin workspace can help us organize one or more (ROS) packages.
Essentially catkin workspace use CMake build system for building software packages. Any CMake-compliant package contains one or more CMakeLists.txt file that describe how to build the code and where to install it to. 

References:
* http://wiki.ros.org/catkin/Tutorials/create_a_workspace 
* http://wiki.ros.org/catkin/CMakeLists.txt

In [ ]:
!mkdir -p /content/catkin_ws/src
%cd /content/catkin_ws
!source /opt/ros/melodic/setup.bash && catkin_make

After running catkin_make, you will expected to see a build and a devel folder under catkin_ws.

### Environment Steup

We have seen that we need to source /opt/ros/melodic/setup.bash in a command. To use the packages we have in the catkin_ws, we could do it by sourcing /content/catkin_ws/devel/setup.bash. This will also automatically source /opt/ros/melodic/setup.bash for us.

In [ ]:
!source /content/catkin_ws/devel/setup.bash

Sometimes if you have a large project with more than one catkin workspace, we could setup a environment.sh

In [ ]:
%%writefile /content/environment.sh

source /opt/ros/melodic/setup.bash 
source /content/catkin_ws/devel/setup.bash

In [ ]:
%cd /content/
!source environment.sh

## B. ROS package
Package contains our nodes, launch files, some libraries, etc. ROS will first find the package in the package list and then find the files in your package to run.


### 1. catkin_create_pkg

We can create the template package by use the catkin_create_pkg script

**!catkin_create_pkg <package_name> [depend1] [depend2] [depend3]** 

at the folder which locates at /catkin_ws/src. 


In [ ]:
%cd /content/catkin_ws/src
!source /content/environment.sh && catkin_create_pkg rospackage_first_example roscpp rospy

In [ ]:
%cd /content/catkin_ws/
!source /content/environment.sh && catkin_make

In [ ]:
%cd /content/catkin_ws
!source /content/environment.sh && rospack find rospackage_first_example

### 2. Clone an example ROS package from Github

This package includes ros service and customized ros message.


In [ ]:
%cd /content/catkin_ws/src/
#!rm -rf rospackage_example # remove if needed 
!git clone https://github.com/Sensing-Intelligent-System/rospackage_example.git

Then, we need to build the packages in the catkin workspace.

In [ ]:
%cd /content/catkin_ws
!source /content/environment.sh && catkin_make

In [ ]:
%cd /content/catkin_ws
!source /content/environment.sh && rospack find rospackage_example

## C. ROS Launch

**roslaunch** is a tool for easily launching multiple ROS nodes locally and remotely via SSH, as well as setting parameters on the Parameter Server. It includes options to automatically respawn processes that have already died. roslaunch takes in one or more XML configuration files (with the .launch extension) that specify the parameters to set and nodes to launch, as well as the machines that they should be run on.

Launch files allow us to run multiple files, dependencies, and ROS master in one command. All the launch file will be written in xml format.


### launch file

In [ ]:
!cat /content/catkin_ws/src/rospackage_example/launch/launch_package.launch

roslaunch will automatically start a roscore. In this exmaple, it will run talker and listener. In Colab we need to make the python files runnable.

In [ ]:
!chmod +x /content/catkin_ws/src/rospackage_example/src/*.py 

In [ ]:
import time
get_ipython().system_raw('source /opt/ros/melodic/setup.bash && roscore & ') 
time.sleep(3)
get_ipython().system_raw('source /content/environment.sh  && roslaunch rospackage_example launch_package.launch & ') 
time.sleep(3)

In [ ]:
!source /content/environment.sh && rosnode list

In [ ]:
!source /content/environment.sh && rostopic list

In [ ]:
%cd /content/
!ls
!chmod +x /content/catkin_ws/src/rospackage_example/src/add_two_ints_server.py
!echo ""
!source /content/environment.sh && printenv | grep ROS
# !source /content/environment.sh && roscd rospackage_example
# !ls
# !echo ""
# !source /content/catkin_ws/devel/setup.bash && rosrun rospackage_example add_two_ints_server

### remap

Remapping allows you to "trick" a ROS node so that when it thinks it is subscribing to or publishing to /some_topic it is actually subscribing to or publishing to /some_other_topic

```
<remap from="/different_topic" to="/needed_topic"/>
```


In [ ]:
!cat /content/catkin_ws/src/rospackage_example/launch/launch_remap.launch

In [ ]:
get_ipython().system_raw('source /content/environment.sh  && roslaunch rospackage_example launch_remap.launch & ') 
time.sleep(3)

In [ ]:
!source /content/environment.sh && rosnode list

In [ ]:
!source /content/environment.sh && rostopic list

## D. ROS Service

See ROS service vs. topic comparisons. http://wiki.ros.org/ROS/Patterns/Communication

### Define your service type, 

The .srv file consists of a request and a response msg type, separated by '- - -'.


In [ ]:
!cat /content/catkin_ws/src/rospackage_example/srv/AddTwoInts.srv

### CMakeList.txt

In [ ]:
!cat /content/catkin_ws/src/rospackage_example/CMakeLists.txt

### rossrv list

In [ ]:
!source /content/environment.sh && rossrv list |grep AddTwoInts

### Server

In [ ]:
!cat /content/catkin_ws/src/rospackage_example/src/add_two_ints_server.py

Start the server

In [ ]:
get_ipython().system_raw('source /content/environment.sh && rosrun rospackage_example add_two_ints_server.py & ')

### rosservice call

In [ ]:
!source /content/environment.sh && rosservice call /add_two_ints 9 10 

## E. ROS Diagram

It is important to document the nodes, messages etc, especially when the system goes larger or more complex.

### graphviz

In [ ]:
from graphviz import Digraph, Graph
dot = Digraph(comment='RosDiagram') # pdf


dot.node('Pub', label = 'publisher: talker')
dot.node('Sub', label = 'listener')

dot.node('Client', label = 'client: add_two_ints_client') 
dot.node('Server', label = 'server: add_two_ints_server') 


dot.edge("Pub", "Sub", label = "  message  ")

dot.edge("Client", "Server", label = "  request  ")
dot.edge("Server", "Client", label = "  response  ")


dot

  

Write to a file.

In [ ]:
dot.format = 'png'
dot.render('/content/catkin_ws/img/diagram', view=True)